In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [23]:
data = pd.read_feather("train.f")

# Prequential Expanding

In [24]:
data['block'] = np.trunc(data['era']*.1).astype(int)
data.loc[data['block'] == 12, 'block'] = 11

In [26]:
data['block'].value_counts().sort_index()

0     24515
1     34600
2     37444
3     41101
4     43439
5     48186
6     46831
7     40403
8     43971
9     45609
10    46107
11    49602
Name: block, dtype: int64

In [34]:
results_val = []

for block in range(1,12):
    print("Train blocks 0-{} - Validation Block {}".format(block - 1, block))
    

    train = data[data['block'] < block]
    val = data[data['block'] == block]
    
    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target_kazutsugi']
    y_val = val['target_kazutsugi']
     

    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()
   

Train blocks 0-10 - Validation Block 11
Correlation 0.04476211397290722



In [35]:
np.median(results_val)

0.04487812028770462

In [36]:
np.min(results_val)

0.02336151215739744

In [37]:
np.max(results_val)

0.06418466180825347

In [38]:
np.mean(results_val)

0.04750744858324451

In [39]:
len(results_val)

11

# Prequential Expanding with Gap
- gap original - remover apenas eras mais próxima do treino, parâmetro = quantas eras remover?

In [40]:
results_val = []

for block in range(2,12):
    print("Train blocks 0-{} - Gap Block {} - Validation Block {}".format(block - 2, block - 1,  block))
    

    train = data[data['block'] < block-1]
    val = data[data['block'] == block]
    
    X_train = train.filter(regex=r'feature')
    X_val = val.filter(regex=r'feature')

    y_train = train['target_kazutsugi']
    y_val = val['target_kazutsugi']
     

    mdl = LGBMRegressor(max_depth=5, num_leaves=2**5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, random_state=0)
    mdl.fit(X_train, y_train)
    
    predictions = pd.Series(mdl.predict(X_val))
    ranked_predictions = predictions.rank(pct=True, method="first")
    correlation = np.corrcoef(y_val, ranked_predictions)[0, 1]
    #print(correlation)
    
    results_val.append(correlation)
    print("Correlation {}".format(correlation))
    print()
   

Train blocks 0-0 - Gap Block 1 - Validation Block 2
Correlation 0.05345227304313937

Train blocks 0-1 - Gap Block 2 - Validation Block 3
Correlation 0.05040186833627322

Train blocks 0-2 - Gap Block 3 - Validation Block 4
Correlation 0.04661229121919876

Train blocks 0-3 - Gap Block 4 - Validation Block 5
Correlation 0.030978313031068485

Train blocks 0-4 - Gap Block 5 - Validation Block 6
Correlation 0.01865208331344953

Train blocks 0-5 - Gap Block 6 - Validation Block 7
Correlation 0.05757610370235812

Train blocks 0-6 - Gap Block 7 - Validation Block 8
Correlation 0.04033177837241513

Train blocks 0-7 - Gap Block 8 - Validation Block 9
Correlation 0.06276979356054141

Train blocks 0-8 - Gap Block 9 - Validation Block 10
Correlation 0.04113250552613722

Train blocks 0-9 - Gap Block 10 - Validation Block 11
Correlation 0.04157351090598695



In [41]:
np.median(results_val)

0.04409290106259286

In [42]:
np.min(results_val)

0.01865208331344953

In [43]:
np.max(results_val)

0.06276979356054141

In [44]:
np.mean(results_val)

0.044348052101056815

In [45]:
len(results_val)

10